In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [16]:
FILE_NAME = "Mental_Health_and_Social_Media_Balance_Dataset.csv"

# Colunas de entrada para o modelo
FEATURES = [
    'Daily_Screen_Time(hrs)',
    'Days_Without_Social_Media',
    'Exercise_Frequency(week)'
]

# Colunas de saída (o que queremos prever)
TARGETS = [
    'Stress_Level(1-10)',
    'Sleep_Quality(1-10)'
]

In [17]:
df = pd.read_csv(FILE_NAME)
print(f"Arquivo '{FILE_NAME}' carregado com sucesso.")
print(f"Total de linhas carregadas: {len(df)}")

Arquivo 'Mental_Health_and_Social_Media_Balance_Dataset.csv' carregado com sucesso.
Total de linhas carregadas: 500


In [18]:
df.head()

,User_ID,Age,Gender,Daily_Screen_Time(hrs),Sleep_Quality(1-10),Stress_Level(1-10),Days_Without_Social_Media,Exercise_Frequency(week),Social_Media_Platform,Happiness_Index(1-10)
0,U001,44,Male,3.1,7.0,6.0,2.0,5.0,Facebook,10.0
1,U002,30,Other,5.1,7.0,8.0,5.0,3.0,LinkedIn,10.0
2,U003,23,Other,7.4,6.0,7.0,1.0,3.0,YouTube,6.0
3,U004,36,Female,5.7,7.0,8.0,1.0,1.0,TikTok,8.0
4,U005,34,Female,7.0,4.0,7.0,5.0,1.0,X (Twitter),8.0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   User_ID                    500 non-null    object 
 1   Age                        500 non-null    int64  
 2   Gender                     500 non-null    object 
 3   Daily_Screen_Time(hrs)     500 non-null    float64
 4   Sleep_Quality(1-10)        500 non-null    float64
 5   Stress_Level(1-10)         500 non-null    float64
 6   Days_Without_Social_Media  500 non-null    float64
 7   Exercise_Frequency(week)   500 non-null    float64
 8   Social_Media_Platform      500 non-null    object 
 9   Happiness_Index(1-10)      500 non-null    float64
dtypes: float64(6), int64(1), object(3)
memory usage: 39.2+ KB


In [20]:
all_cols = FEATURES + TARGETS
df_clean = df[all_cols].copy()

print(f"Total de linhas antes da limpeza: {len(df_clean)}")

Total de linhas antes da limpeza: 500


In [21]:
# Passo 1: Inspecionar dados mistos em 'Daily_Screen_Time(hrs)'
# Esta coluna tem números e, aparentemente, datas (ex: '2025-01-03')
# Vamos forçar tudo para número. O que não for número virará 'NaN' (Not a Number).
print("\nLimpando a coluna 'Daily_Screen_Time(hrs)'...")
df_clean['Daily_Screen_Time(hrs)'] = pd.to_numeric(df_clean['Daily_Screen_Time(hrs)'], errors='coerce')


Limpando a coluna 'Daily_Screen_Time(hrs)'...


In [22]:
# Passo 2: Remover todas as linhas que contenham QUALQUER valor 'NaN'
# Isso remove tanto os dados que já faltavam quanto as datas que falharam na conversão
rows_before_drop = len(df_clean)
df_clean = df_clean.dropna()
rows_after_drop = len(df_clean)

print(f"Linhas removidas por dados inválidos/faltantes: {rows_before_drop - rows_after_drop}")
print(f"Total de linhas limpas e prontas para o modelo: {rows_after_drop}")

Linhas removidas por dados inválidos/faltantes: 0
Total de linhas limpas e prontas para o modelo: 500


In [23]:
print("\n--- Informações dos Dados Pós-Limpeza ---")
df_clean.info()


--- Informações dos Dados Pós-Limpeza ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Daily_Screen_Time(hrs)     500 non-null    float64
 1   Days_Without_Social_Media  500 non-null    float64
 2   Exercise_Frequency(week)   500 non-null    float64
 3   Stress_Level(1-10)         500 non-null    float64
 4   Sleep_Quality(1-10)        500 non-null    float64
dtypes: float64(5)
memory usage: 19.7 KB


In [24]:
print("\n--- Amostra dos Dados Pós-Limpeza ---")
print(df_clean.head())


--- Amostra dos Dados Pós-Limpeza ---
   Daily_Screen_Time(hrs)  Days_Without_Social_Media  \
0                     3.1                        2.0   
1                     5.1                        5.0   
2                     7.4                        1.0   
3                     5.7                        1.0   
4                     7.0                        5.0   

   Exercise_Frequency(week)  Stress_Level(1-10)  Sleep_Quality(1-10)  
0                       5.0                 6.0                  7.0  
1                       3.0                 8.0                  7.0  
2                       3.0                 7.0                  6.0  
3                       1.0                 8.0                  7.0  
4                       1.0                 7.0                  4.0  


In [25]:
# X = Dados de entrada (as causas)
# Y = Dados de saída (os efeitos que queremos prever)

X = df_clean[FEATURES]
Y = df_clean[TARGETS]

In [26]:
print("--- Features (X) ---")
print(X.head())
print(f"Formato de X: {X.shape}")

print("\n--- Targets (Y) ---")
print(Y.head())
print(f"Formato de Y: {Y.shape}")

--- Features (X) ---
   Daily_Screen_Time(hrs)  Days_Without_Social_Media  Exercise_Frequency(week)
0                     3.1                        2.0                       5.0
1                     5.1                        5.0                       3.0
2                     7.4                        1.0                       3.0
3                     5.7                        1.0                       1.0
4                     7.0                        5.0                       1.0
Formato de X: (500, 3)

--- Targets (Y) ---
   Stress_Level(1-10)  Sleep_Quality(1-10)
0                 6.0                  7.0
1                 8.0                  7.0
2                 7.0                  6.0
3                 8.0                  7.0
4                 7.0                  4.0
Formato de Y: (500, 2)


In [27]:
# Vamos usar 80% dos dados para treinar o modelo e 20% para testá-lo
# random_state=42 garante que a divisão seja sempre a mesma, para resultados reproduzíveis
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Dados divididos em conjuntos de treino e teste:")
print(f"  Tamanho do Treino (X_train): {X_train.shape}")
print(f"  Tamanho do Teste (X_test):   {X_test.shape}")
print(f"  Tamanho do Treino (Y_train): {Y_train.shape}")
print(f"  Tamanho do Teste (Y_test):   {Y_test.shape}")

Dados divididos em conjuntos de treino e teste:
  Tamanho do Treino (X_train): (400, 3)
  Tamanho do Teste (X_test):   (100, 3)
  Tamanho do Treino (Y_train): (400, 2)
  Tamanho do Teste (Y_test):   (100, 2)


In [28]:
# Vamos usar o RandomForestRegressor. Ele é robusto e lida bem com múltiplas saídas.
# n_estimators=100 significa que ele vai construir 100 "árvores de decisão".
# n_jobs=-1 usa todos os processadores disponíveis para acelerar o treino.

model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# O passo mais importante: "aprender" com os dados de treino
model.fit(X_train, Y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [29]:
# Pedir ao modelo para prever os resultados para os dados de teste (que ele nunca viu)
Y_pred = model.predict(X_test)

In [30]:
# Avaliar cada alvo (Estresse e Sono) separadamente
print("\n--- Previsão de 'Nível de Estresse' ---")
mse_stress = mean_squared_error(Y_test['Stress_Level(1-10)'], Y_pred[:, 0])
r2_stress = r2_score(Y_test['Stress_Level(1-10)'], Y_pred[:, 0])

print(f"  Erro Quadrático Médio (MSE): {mse_stress:.2f}")
print(f"  Pontuação R-squared (R²): {r2_stress:.2f}")
print("  (MSE: quanto menor, melhor. R²: quanto mais perto de 1.0, melhor)")


--- Previsão de 'Nível de Estresse' ---
  Erro Quadrático Médio (MSE): 1.14
  Pontuação R-squared (R²): 0.53
  (MSE: quanto menor, melhor. R²: quanto mais perto de 1.0, melhor)


In [31]:
print("\n--- Previsão de 'Qualidade do Sono' ---")
mse_sleep = mean_squared_error(Y_test['Sleep_Quality(1-10)'], Y_pred[:, 1])
r2_sleep = r2_score(Y_test['Sleep_Quality(1-10)'], Y_pred[:, 1])

print(f"  Erro Quadrático Médio (MSE): {mse_sleep:.2f}")
print(f"  Pontuação R-squared (R²): {r2_sleep:.2f}")
print("  (MSE: quanto menor, melhor. R²: quanto mais perto de 1.0, melhor)")


--- Previsão de 'Qualidade do Sono' ---
  Erro Quadrático Médio (MSE): 1.18
  Pontuação R-squared (R²): 0.43
  (MSE: quanto menor, melhor. R²: quanto mais perto de 1.0, melhor)


In [32]:
# O que o modelo achou mais importante para fazer as previsões?
importances = model.feature_importances_

# Criar um DataFrame para visualizar melhor
feature_importance_df = pd.DataFrame({
    'Feature': FEATURES,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)
print("\n(Valores mais altos indicam maior impacto nas previsões)")

                     Feature  Importance
0     Daily_Screen_Time(hrs)    0.759774
1  Days_Without_Social_Media    0.131581
2   Exercise_Frequency(week)    0.108645

(Valores mais altos indicam maior impacto nas previsões)


In [35]:
# Cenário de previsão teste:
# - 6.0 horas de tempo de tela
# - 1 dia sem rede social
# - 2 exercícios por semana
novo_cenario = pd.DataFrame([
    [6.0, 3, 0]
], columns=FEATURES)

print(f"Para os dados de entrada:\n{novo_cenario}\n")

Para os dados de entrada:
   Daily_Screen_Time(hrs)  Days_Without_Social_Media  Exercise_Frequency(week)
0                     6.0                          3                         0



In [36]:
prediction = model.predict(novo_cenario)

print("Previsão do modelo:")
print(f"  Nível de Estresse (1-10): {prediction[0][0]:.1f}")
print(f"  Qualidade do Sono (1-10): {prediction[0][1]:.1f}")

Previsão do modelo:
  Nível de Estresse (1-10): 6.5
  Qualidade do Sono (1-10): 5.5


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


In [38]:
FILE_NAME = "Mental_Health_and_Social_Media_Balance_Dataset.csv"
FEATURES = [
    'Daily_Screen_Time(hrs)',
    'Days_Without_Social_Media',
    'Exercise_Frequency(week)'
]
TARGETS = [
    'Stress_Level(1-10)',
    'Sleep_Quality(1-10)'
]

In [39]:
df = pd.read_csv(FILE_NAME)

In [40]:
df.head()

,User_ID,Age,Gender,Daily_Screen_Time(hrs),Sleep_Quality(1-10),Stress_Level(1-10),Days_Without_Social_Media,Exercise_Frequency(week),Social_Media_Platform,Happiness_Index(1-10)
0,U001,44,Male,3.1,7.0,6.0,2.0,5.0,Facebook,10.0
1,U002,30,Other,5.1,7.0,8.0,5.0,3.0,LinkedIn,10.0
2,U003,23,Other,7.4,6.0,7.0,1.0,3.0,YouTube,6.0
3,U004,36,Female,5.7,7.0,8.0,1.0,1.0,TikTok,8.0
4,U005,34,Female,7.0,4.0,7.0,5.0,1.0,X (Twitter),8.0


In [41]:
all_cols = FEATURES + TARGETS
df_clean = df[all_cols].copy()
rows_before_drop = len(df_clean)

In [42]:
df_clean['Daily_Screen_Time(hrs)'] = pd.to_numeric(df_clean['Daily_Screen_Time(hrs)'], errors='coerce')
df_clean = df_clean.dropna()
rows_after_drop = len(df_clean)

print(f"\nLinhas antes da limpeza: {rows_before_drop}")
print(f"Linhas removidas por dados inválidos: {rows_before_drop - rows_after_drop}")
print(f"Total de linhas limpas (dataset original): {rows_after_drop}")


Linhas antes da limpeza: 500
Linhas removidas por dados inválidos: 0
Total de linhas limpas (dataset original): 500


In [43]:
if rows_after_drop < 50: # Um limite arbitrário
    print("\nAVISO: Poucos dados originais restaram. Aumentar os dados é uma boa ideia.")

print("\n--- Amostra dos Dados Pós-Limpeza ---")
print(df_clean.head())


--- Amostra dos Dados Pós-Limpeza ---
   Daily_Screen_Time(hrs)  Days_Without_Social_Media  \
0                     3.1                        2.0   
1                     5.1                        5.0   
2                     7.4                        1.0   
3                     5.7                        1.0   
4                     7.0                        5.0   

   Exercise_Frequency(week)  Stress_Level(1-10)  Sleep_Quality(1-10)  
0                       5.0                 6.0                  7.0  
1                       3.0                 8.0                  7.0  
2                       3.0                 7.0                  6.0  
3                       1.0                 8.0                  7.0  
4                       1.0                 7.0                  4.0  


In [44]:
# Aumento de Dados (Data Augmentation)
# O SMOTE é para classificação. Para regressão, usaremos re-amostragem com ruído.
original_data = df_clean.copy()
original_size = len(original_data)

In [76]:
# Vamos criar 9x mais dados sintéticos para ter um total 10x maior
augmentation_factor = 500
n_new_samples = original_size * (augmentation_factor - 1)
print(f"Tamanho original: {original_size}. Gerando {n_new_samples} novas amostras.")

Tamanho original: 500. Gerando 249500 novas amostras.


In [77]:
# Usaremos 15% (0.15) do desvio padrão como a intensidade do ruído
noise_level = 0.15
noise_std = original_data.std() * noise_level
print("\nNível de ruído (15% do std dev de cada coluna):")
print(noise_std)


Nível de ruído (15% do std dev de cada coluna):
Daily_Screen_Time(hrs)       0.260232
Days_Without_Social_Media    0.278813
Exercise_Frequency(week)     0.214210
Stress_Level(1-10)           0.231449
Sleep_Quality(1-10)          0.229469
dtype: float64


In [78]:
new_samples_list = []
for _ in range(n_new_samples):
    # Pega uma amostra aleatória do dataset original
    random_sample = original_data.sample(n=1).iloc[0]

    # Cria ruído aleatório baseado no desvio padrão de cada coluna
    noise = np.random.normal(loc=0.0, scale=noise_std, size=len(noise_std))
    noise_series = pd.Series(noise, index=noise_std.index)

    # Adiciona o ruído à amostra
    new_sample = random_sample + noise_series

    # 4. Pós-processamento: Garantir que os dados façam sentido
    # Não podemos ter valores negativos ou fora dos limites (ex: Estresse 1-10)

    # 'Daily_Screen_Time(hrs)': Mínimo 0, Máximo 24
    new_sample['Daily_Screen_Time(hrs)'] = np.clip(new_sample['Daily_Screen_Time(hrs)'], 0, 24)

    # 'Days_Without_Social_Media': Mínimo 0, Máximo 7 (e arredondado)
    new_sample['Days_Without_Social_Media'] = np.round(np.clip(new_sample['Days_Without_Social_Media'], 0, 7))

    # 'Exercise_Frequency(week)': Mínimo 0, Máximo 7 (e arredondado)
    new_sample['Exercise_Frequency(week)'] = np.round(np.clip(new_sample['Exercise_Frequency(week)'], 0, 7))

    # 'Stress_Level(1-10)': Mínimo 1, Máximo 10 (e arredondado)
    new_sample['Stress_Level(1-10)'] = np.round(np.clip(new_sample['Stress_Level(1-10)'], 1, 10))

    # 'Sleep_Quality(1-10)': Mínimo 1, Máximo 10 (e arredondado)
    new_sample['Sleep_Quality(1-10)'] = np.round(np.clip(new_sample['Sleep_Quality(1-10)'], 1, 10))

    new_samples_list.append(new_sample)

In [79]:
#Combinar dados originais e sintéticos
new_samples_df = pd.DataFrame(new_samples_list, columns=original_data.columns)
df_augmented = pd.concat([original_data, new_samples_df], ignore_index=True)

print(f"\nDados aumentados com sucesso!")
print(f"Tamanho do novo dataset (original + sintético): {len(df_augmented)}")

# Mostrar as últimas linhas (que são sintéticas)
print("\n--- Amostra dos Dados Sintéticos Gerados (Tail) ---")
print(df_augmented.tail())


Dados aumentados com sucesso!
Tamanho do novo dataset (original + sintético): 250000

--- Amostra dos Dados Sintéticos Gerados (Tail) ---
        Daily_Screen_Time(hrs)  Days_Without_Social_Media  \
249995                7.878395                        3.0   
249996                8.443218                        0.0   
249997                7.183166                        5.0   
249998                6.437554                        6.0   
249999                6.725697                        5.0   

        Exercise_Frequency(week)  Stress_Level(1-10)  Sleep_Quality(1-10)  
249995                       3.0                 8.0                  5.0  
249996                       2.0                10.0                  5.0  
249997                       4.0                 6.0                  6.0  
249998                       0.0                 7.0                  6.0  
249999                       5.0                 9.0                  4.0  


In [80]:
X = df_augmented[FEATURES]
Y = df_augmented[TARGETS]

print("\n--- Features (X) do dataset aumentado ---")
print(X.head())
print(f"Formato de X: {X.shape}")

print("\n--- Targets (Y) do dataset aumentado ---")
print(Y.head())
print(f"Formato de Y: {Y.shape}")


--- Features (X) do dataset aumentado ---
   Daily_Screen_Time(hrs)  Days_Without_Social_Media  Exercise_Frequency(week)
0                     3.1                        2.0                       5.0
1                     5.1                        5.0                       3.0
2                     7.4                        1.0                       3.0
3                     5.7                        1.0                       1.0
4                     7.0                        5.0                       1.0
Formato de X: (250000, 3)

--- Targets (Y) do dataset aumentado ---
   Stress_Level(1-10)  Sleep_Quality(1-10)
0                 6.0                  7.0
1                 8.0                  7.0
2                 7.0                  6.0
3                 8.0                  7.0
4                 7.0                  4.0
Formato de Y: (250000, 2)


In [81]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05, random_state=42)

print("\nDados (aumentados) divididos em conjuntos de treino e teste:")
print(f"  Tamanho do Treino (X_train): {X_train.shape}")
print(f"  Tamanho do Teste (X_test):   {X_test.shape}")


Dados (aumentados) divididos em conjuntos de treino e teste:
  Tamanho do Treino (X_train): (237500, 3)
  Tamanho do Teste (X_test):   (12500, 3)


In [82]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, Y_train)
print("Modelo treinado com sucesso!")

Modelo treinado com sucesso!


In [83]:
Y_pred = model.predict(X_test)

In [84]:
print("\n--- Previsão de 'Nível de Estresse' ---")
mse_stress = mean_squared_error(Y_test['Stress_Level(1-10)'], Y_pred[:, 0])
r2_stress = r2_score(Y_test['Stress_Level(1-10)'], Y_pred[:, 0])
print(f"  Erro Quadrático Médio (MSE): {mse_stress:.2f} - quanto menor, melhor")
print(f"  Pontuação R-squared (R²): {r2_stress:.2f} (Mais perto de 1.0 é melhor)")


--- Previsão de 'Nível de Estresse' ---
  Erro Quadrático Médio (MSE): 0.99 - quanto menor, melhor
  Pontuação R-squared (R²): 0.60 (Mais perto de 1.0 é melhor)


In [85]:
print("\n--- Previsão de 'Qualidade do Sono' ---")
mse_sleep = mean_squared_error(Y_test['Sleep_Quality(1-10)'], Y_pred[:, 1])
r2_sleep = r2_score(Y_test['Sleep_Quality(1-10)'], Y_pred[:, 1])
print(f"  Erro Quadrático Médio (MSE): {mse_sleep:.2f}")
print(f"  Pontuação R-squared (R²): {r2_sleep:.2f} (Mais perto de 1.0 é melhor)")


--- Previsão de 'Qualidade do Sono' ---
  Erro Quadrático Médio (MSE): 0.98
  Pontuação R-squared (R²): 0.59 (Mais perto de 1.0 é melhor)


In [86]:
print("\n--- Importância das Features ---")
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': FEATURES,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importance_df)


--- Importância das Features ---
                     Feature  Importance
0     Daily_Screen_Time(hrs)    0.851201
1  Days_Without_Social_Media    0.081649
2   Exercise_Frequency(week)    0.067150


In [89]:
# Cenário:
# - 6.0 horas de tempo de tela
# - 1 dia sem rede social
# - 2 exercícios por semana
novo_cenario = pd.DataFrame([
    [2.0, 0, 5]
], columns=FEATURES)

In [90]:
print(f"Para os dados de entrada:\n{novo_cenario}\n")
prediction = model.predict(novo_cenario)

print("Previsão do modelo (treinado com dados aumentados):")
print(f"  Nível de Estresse (1-10): {prediction[0][0]:.1f}")
print(f"  Qualidade do Sono (1-10): {prediction[0][1]:.1f}")

Para os dados de entrada:
   Daily_Screen_Time(hrs)  Days_Without_Social_Media  Exercise_Frequency(week)
0                     2.0                          0                         5

Previsão do modelo (treinado com dados aumentados):
  Nível de Estresse (1-10): 3.5
  Qualidade do Sono (1-10): 9.4
